# Data Preprocessing

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import RobustScaler

In [2]:
data = pd.read_csv('cleanest_data.csv')

## ДОСЛІДЖЕННЯ РОЗПОДІЛУ

*Перевірка на асиметрію*:

In [3]:
data.skew(numeric_only=True)

is_successful                    -0.542771
order_amount                     63.597978
order_messages                    5.475379
order_changes                     8.201835
partner_success_rate             -0.708427
partner_total_orders              3.626058
partner_order_age_days            0.570445
partner_avg_amount               92.806970
partner_success_avg_amount       18.085258
partner_fail_avg_amount          46.378813
partner_total_messages            3.568156
partner_success_avg_messages      1.976863
partner_fail_avg_messages         4.583495
partner_avg_changes               4.765490
partner_success_avg_changes       2.553038
partner_fail_avg_changes          7.842901
quarter                           0.015067
hour_of_day                       0.331366
order_lines_count               175.787705
discount_total                  113.972894
dtype: float64

### Статистичні тести нормальності розподілу
**Тест Шапіро-Уілка** (scipy.stats.shapiro)
- Дуже точний, але для малих вибірок (до 5000 записів).

**Тест Д'Агостіно-Кільмейра** (scipy.stats.normaltest)
- Підходить для великих вибірок.

**Тест Андерсона-Дарлінга** (scipy.stats.anderson)
- Ще один строгий тест для нормальності.

In [4]:
from scipy.stats import normaltest, anderson

In [5]:
# Вибираємо тільки числові колонки
numeric_cols = data.select_dtypes(include=[np.number]).columns

# Створюємо результуючу таблицю
results = []

for col in numeric_cols:
    col_data = data[col]

    # Тест Д'Агостіно-Кільмейра
    stat_dagostino, p_dagostino = normaltest(col_data)

    # Тест Андерсона-Дарлінга
    result_anderson = anderson(col_data)
    stat_anderson = result_anderson.statistic
    critical_anderson = result_anderson.critical_values[2]  # поріг для рівня значущості 5%

    # Оцінка нормальності
    is_normal_dagostino = p_dagostino > 0.05
    is_normal_anderson = stat_anderson < critical_anderson

    results.append({
        'column': col,
        'dagostino_stat': stat_dagostino,
        'dagostino_p': p_dagostino,
        'dagostino_is_normal': is_normal_dagostino,
        'anderson_stat': stat_anderson,
        'anderson_critical_5%': critical_anderson,
        'anderson_is_normal': is_normal_anderson
    })

# Перетворюємо в DataFrame для красивого вигляду
normality_results = pd.DataFrame(results)

In [6]:
normality_results

,column,dagostino_stat,dagostino_p,dagostino_is_normal,anderson_stat,anderson_critical_5%,anderson_is_normal
0,is_successful,3.747188e+05,0.0,False,17025.011072,0.787,False
1,order_amount,2.970682e+05,0.0,False,26444.435265,0.787,False
2,order_messages,9.640125e+04,0.0,False,7509.998823,0.787,False
3,order_changes,1.278144e+05,0.0,False,7624.119674,0.787,False
4,partner_success_rate,8.396103e+03,0.0,False,2632.940401,0.787,False
5,partner_total_orders,6.608209e+04,0.0,False,11546.879620,0.787,False
6,partner_order_age_days,1.044213e+04,0.0,False,1867.569814,0.787,False
7,partner_avg_amount,3.370831e+05,0.0,False,22123.840104,0.787,False
8,partner_success_avg_amount,1.818324e+05,0.0,False,13989.165778,0.787,False
9,partner_fail_avg_amount,2.666633e+05,0.0,False,21310.127781,0.787,False


Поояснення показників:

| Показник           | Що означає                                                               |
| :----------------- | :----------------------------------------------------------------------- |
| dagostino\_stat    | Статистика тесту Д’Агостіно-Кільмейра (скошеність + ексцес).               |
| dagostino\_p       | P-значення тесту Д’Агостіно-Кільмейра.                                   |
| dagostino\_is\_normal | Чи приймаємо нормальність за результатами Д’Агостіно-Кільмейра?         |
| anderson\_stat     | Статистика тесту Андерсона-Дарлінга.                                     |
| anderson\_critical\_5% | Критичне значення тесту Андерсона-Дарлінга при рівні 5%.                |
| anderson\_is\_normal | Чи приймаємо нормальність за результатами Андерсона-Дарлінга?           |

**Висновок:** *всі данні мають не нормальний розподіл*

### ДОСЛІДЖЕННЯ ЧИСЛОВИХ СТАТИСТИК

In [ ]:
def numeric_statistic(data):
    # 1. Виділяємо тільки числові колонки
    numeric_columns = data.select_dtypes(include=['number']).columns

    # 2. Створюємо список для зберігання результатів
    stats_list = []

    # 3. Обходимо всі числові колонки
    for col in numeric_columns:
        col_min = data[col].min()
        col_max = data[col].max()
        col_mean = data[col].mean()
        col_median = data[col].median()
        # mode() може повертати кілька значень, беремо перше
        col_mode = data[col].mode().iloc[0] if not data[col].mode().empty else None

        stats_list.append({
            'column': col,
            'min': col_min,
            'max': col_max,
            'mean': col_mean,
            'median': col_median,
            'mode': col_mode
        })

    # 4. Створюємо фінальний DataFrame
    stats_df = pd.DataFrame(stats_list)
    return stats_df

In [ ]:
statistic = numeric_statistic(data)